In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image

# sklearn utilities
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# sklearn models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA

# Mount drive and build path (only necessary if using Google Colab)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
# import os

# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS3540/Final-Project' # change this directory to yours
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(GOOGLE_DRIVE_PATH)
# print(os.listdir(GOOGLE_DRIVE_PATH))

# Load image pixel values into array as features and create target array

In [ ]:
# Adds fire images to array
images = np.zeros((2000, 250, 250, 3))
j = 0
for i in range(1522):
  try:
    image_name = 'F_' + str(i) + '.jpg'
    path = "Fire/" + image_name
    image = (Image.open(path))
    img = np.asarray(image)
    images[j] = np.copy(img)
    j = j+1
  except:
    continue

In [ ]:
# Adds non-fire images to array
for i in range(1011):
  try:
    image_name = 'NF_' + str(i) + '.jpg'
    path = 'Non_Fire/' + image_name
    image = (Image.open(path))
    img = np.asarray(image)
    images[j] = np.copy(img)
    j = j+1
  except:
    continue

In [ ]:
# Create targets array which maps to features array
targets = np.zeros(2000)
for i in range(1000):
  targets[i] = 1

In [ ]:
  # Only run this if you have .npz file saved - make sure path is correct
  # <15 seconds to load
# images = np.load('images.npz')['data']

# Preprocess data and apply random split

In [ ]:
# Flatten and normalize features array for use with PCA
images_flat = images.reshape((2000, 187500))
images_norm = normalize(images_flat)

In [ ]:
# Apply PCA feature reduction
pca = PCA(10)
transformed_data = pca.fit_transform(images_norm)

# Split data and plot scatter plot
# plot_x, plot_y = np.hsplit(transformed_data, 2)
# scatter = plt.scatter(plot_x, plot_y, c=targets)
# plt.legend(*scatter.legend_elements(), loc='upper left')

In [ ]:
# Apply train/test split of 75/25
x_train, x_test, y_train, y_test = train_test_split(transformed_data, targets, test_size=.25)
x_train_cnn, x_test_cnn, y_train_cnn, y_test_cnn = train_test_split(images_norm, targets, test_size=.25)

In [ ]:
# Confirm shape of features and targets
print("Training x:", x_train.shape,"y:", y_train.shape)
print("Testing x:", x_test.shape,"y:", y_test.shape)

# Logistic Regression

In [ ]:
# Run Logistic Regression model
logreg_model = LogisticRegression()
logreg_model.fit(x_train, y_train)
logreg_results = logreg_model.predict(x_test)

In [ ]:
# Build and display confusion matrix
logreg_conf_matrix = confusion_matrix(y_test, logreg_results)
pd.DataFrame(logreg_conf_matrix, columns=[0, 1], index=[0,1])

In [ ]:
# Run classification report on Logistic Regression results
print(classification_report(y_test, logreg_results))

# SVM

In [ ]:
# Run SVM model
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train, y_train)
svm_results = svm_model.predict(x_test)

In [ ]:
# Build and display confusion matrix
svm_conf_matrix = confusion_matrix(y_test, svm_results)
pd.DataFrame(svm_conf_matrix, columns=[0, 1], index=[0,1])

In [ ]:
# Run classification report on SVM results
print(classification_report(y_test, svm_results))

# CNN

In [ ]:
# Reshape features to original dimensions
x_train_cnn = np.reshape(x_train_cnn, (1500, 250, 250, 3))
y_train_cnn = np.reshape(y_train_cnn, (1500, 1)) # is this necessary?
x_test_cnn = np.reshape(x_test_cnn, (500, 250, 250, 3))
y_test_cnn = np.reshape(y_test_cnn, (500, 1)) # is this necessary?

In [ ]:
# Apply train/validation split of 80/20
xtrain, xval, ytrain, yval = train_test_split(x_train_cnn, y_train_cnn, test_size=0.20)

In [ ]:
# Build CNN model
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(250, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2))

# Display model summary
model.summary()

In [ ]:
# Compile and train model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(xtrain, ytrain, epochs=15, validation_data=(xval, yval))

In [ ]:
# Plot training and validation accuracy graph over epochs
plt.plot(history.history['accuracy'], label="accuracy")
plt.plot(history.history['val_accuracy'], label="val_accuracy")
plt.legend()
plt.show()

In [ ]:
# Plot training and validation accuracy graph over epochs
plt.plot(history.history['loss'], label="loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.legend()
plt.show()

In [ ]:
# Evaluate model on test data
model.evaluate(x_test_cnn, y_test_cnn)